In [1]:
# !python -m pip install --upgrade pip
# !python -m pip install --upgrade pyspark

# !pip install msgpack
# !pip install ipython
#!pip install pyspark==2.3.0
# !pip install pyLDAvis
# !pip install ipynb

In [2]:
import os
# os.environ["JAVA_HOME"] = "C:\Java\jdk1.8.0_172"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[2] pyspark-shell"
os.environ["JAVA_HOME"] = "C:/jdk1.8.0_171"
#USATO PER RISOLVEREE PROBLEMA JAVA GATEWAY ..... (ho anche installato java e inserito manualmente java home tra le variabili ambiente)

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
from pyspark.sql.functions import monotonically_increasing_id
import tempfile
import re
import json
import pandas as pd
import nltk
from nltk.corpus import stopwords
import pyspark
import string
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.mllib.util import MLUtils
from pyspark.sql.types import *
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
nltk.download("stopwords")
import pyLDAvis
import myLDAfunctions as mylib 
from imp import reload

[nltk_data] Downloading package stopwords to C:\Users\Roberta
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
reload(mylib) 

<module 'myLDAfunctions' from 'C:\\Users\\Roberta PC\\Desktop\\Università\\LDA\\latentDirichletAllocation\\FlaskApp\\myLDAfunctions.py'>

In [5]:
spark = SparkSession.builder.appName('topicModelLDA').getOrCreate()
sc = spark.sparkContext

In [6]:
filename = "ARTICOLITV2.json"
multiline = True
df = mylib.read_our_json_dataset(filename, multiline)
clean_text, stopwords = mylib.pulizia_df_train(df, text_col = "testo_txt_it", number_of_words =100)

In [7]:
train, test = clean_text.select("final").randomSplit([0.95, 0.05])


In [8]:
df.show(5, truncate = False)

+-------------------+----------------------------------------+--------+----------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
clean_text.show(5, truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
cvmodel, cvDatasetTrain, idfModel, rescaledDataTrain = mylib.tf_idf_train (train)
cvDatasetTest, rescaledDataTest = mylib.tf_idf_test (test, cvmodel, idfModel)


In [12]:
k = 10 #numero topic
lda = LDA(k=k, seed=123, optimizer="em",featuresCol="features")
ldamodel = lda.fit(rescaledDataTrain)


# The Optimizer implements the Online variational Bayes LDA algorithm,
# which processes a subset of the corpus on each iteration,
# and updates the term-topic distribution adaptively.



In [18]:
k = 10 #numero topic
ldaONLINE = LDA(k=k, seed=123, optimizer="online",featuresCol="features")
ldamodelOnline = ldaONLINE.fit(rescaledDataTrain)


In [13]:
predictionTrain = ldamodel.transform(rescaledDataTrain)
prediction = ldamodel.transform(rescaledDataTest)

In [19]:
predictionTrainOnline = ldamodel.transform(rescaledDataTrain)
predictionOnline = ldamodel.transform(rescaledDataTest)

In [14]:
vocab, doc_lengths, term_frequency, topic_term_dists, doc_topic_dists = mylib.extract_data (ldamodel, cvDatasetTrain, cvmodel, train, predictionTrain, "rawFeatures")
prep_visual = pyLDAvis.prepare(topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency )
pyLDAvis.display(prep_visual) 

C:\Anac\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


In [20]:
vocabOnline, doc_lengthsOnline, term_frequencyOnline, topic_term_distsOnline, doc_topic_distsOnline = mylib.extract_data (ldamodelOnline, cvDatasetTrain, cvmodel, train, predictionTrainOnline, "rawFeatures")
prep_visualOnline = pyLDAvis.prepare(topic_term_distsOnline, doc_topic_distsOnline, doc_lengthsOnline, vocabOnline, term_frequencyOnline )
pyLDAvis.display(prep_visualOnline) #da runnare

In [21]:
pyLDAvis.save_html(prep_visualOnline, "C:/Users/Roberta PC/Desktop/Università/LDA/latentDirichletAllocation/FlaskApp/templates/visualization.html")

In [23]:
print("Train-set results")
print("LogLikelihood", ldamodelOnline.logLikelihood(rescaledDataTrain), "LogPerplexity", ldamodelOnline.logPerplexity(rescaledDataTrain))

Train-set results
LogLikelihood -2050085.0971381129 LogPerplexity 7.561495256553272


In [25]:
print("Test-set results")
print("LogLikelihood", ldamodelOnline.logLikelihood(rescaledDataTest), "LogPerplexity", ldamodelOnline.logPerplexity(rescaledDataTest))

Test-set results
LogLikelihood -136579.16607471197 LogPerplexity 12.37704963883738


In [ ]:
#vocabTest, doc_lengthsTest, term_frequencyTest, topic_term_distsTest, doc_topic_distsTest = extract_data (ldamodel, cvDatasetTest, cvmodel, test, prediction, "rawFeatures")
#prep_visualTest = pyLDAvis.prepare(topic_term_distsTest, doc_topic_distsTest, doc_lengthsTest, vocabTest, term_frequencyTest )
#pyLDAvis.display(prep_visualTest)

In [ ]:
# temp_path = tempfile.mkdtemp()
# ldaModelPath = temp_path + "/ldamodel"
# ldamodel.save(ldaModelPath)
# ldamodel.save("C:/lda_save/provasave")

In [30]:
from flask import Flask, flash, redirect, render_template, request, session, abort
from flask_bootstrap import Bootstrap
from textblob import TextBlob, Word
from flask import Flask, render_template, url_for, request
import numpy as np
import time


In [ ]:
app = Flask (__name__)

@app.route("/")
def index ():
    return render_template("home.html")

@app.route('/prediction',methods=['POST'])
def prediction ():
    if request.method == 'POST':
        rawtext = request.form['rawtext']
        start = time.time()
        string_article = {"testo_txt_it":rawtext}
        jsonRDD = sc.parallelize([json.dumps(string_article)]) 
        df= spark.read.json(jsonRDD) 
        test_clean = mylib.clean_test(dfnew, stopwords, "testo_txt_it")
        cvDatasetTest, rescaledDataTestNew = mylib.tf_idf_test (test_clean, cvmodel, idfModel)
        test_predictionsOnline = ldamodelOnline.transform(rescaledDataTestNew)
        #test_predictions = ldamodel.transform(rescaledDataTestNew)
        finish = time.time()
        time_sentence = ' '.join([str(i) for i in ("Tempo impiegato per la previsione",np.round(finish-start,2))])
        topics_list = test_predictionsOnline.select("topicDistribution").take(1)[0][0]
        topics = {str(i+1): np.round(topics_list[i],2) for i in range(len(topics_list)) if topics_list[i]>=0.10}
        if len(topics.keys())==0:
            topics = "Nessuno degli argomenti definiti dal train è stato trattato nel documento. Provare con un altro articolo."
        else:
            string_tuple = "L'articolo esaminato tratta i seguenti topic :" ,', '.join([str(i) for i in list(topics.keys())]) , "con le rispettive percentuali: ",', '.join([str(i) for i in list(topics.values())])  
            topics = ' '.join([str(i) for i in string_tuple])
        return render_template("home.html", topics = topics, timing = time_sentence )

@app.route('/visualization')
def visualization ():
    return render_template("visualization.html",)
        
@app.route("/slideshow")
def slideshow():
    return render_template("slide_template.html")

if __name__ == "__main__":
    app.run(port = 8889)